<a href="https://colab.research.google.com/github/evelyntr/Cal-Poly-301-lab-6-phase-1/blob/main/Copy_of_DATA_301_Lab_6A_Evelyn_Tran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CityBikes Rest API

This section asks you to fetch JSON data from the [CityBikes REST API](https://api.citybik.es/v2/) to answer some questions about bike share programs.

Please be mindful not to send requests to the server more often than necessary. In particular, make sure you separate code that requests data from the server into a separate cell so that you do not have to repeatedly request data from the server. You will lose points if your requests are not separated into their own cell.

In [ ]:
# Fetch data from a URL
import requests
import pandas as pd
from pandas.io.json import json_normalize

response = requests.get("http://api.citybik.es/v2/networks/")
data = response.json()


# Question 1

Find all cities with bike share programs in the United States (country code "US"), along with their network ID (for example, "ford-gobike"). How many cities in the U.S. have bike share programs (at least that are in this API)?

In [ ]:
rows = []
for program in data['networks']:
  row = {}
  row["country"] = program['location']['country']
  row["city"] = program['location']['city']
  row['networkID'] = program['id']
  row['latitude'] = program['location']['latitude']
  row['longitude'] = program['location']['longitude']
  rows.append(row)
df = pd.DataFrame(rows)
len(df[df['country'] == 'US']['city'].unique())

64

There are 64 citites in the US that have bike share programs. 

# Question 2

Construct a `DataFrame` containing data about all bike stations in all networks in the United States. Save this `DataFrame` to disk using `.to_csv()`. (You will need it in Part B of this lab.)

Use this `DataFrame` to determine the total number of bicycles in bike share programs across the United States? You may assume that the number of bikes at a station is the number of empty spaces, plus the number of available bikes.

In [27]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/citybikes_us.csv'



df_us_bikes = df[df['country'] == 'US']
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_us_bikes.to_csv(f)
df_us_bikes
num_bikes = 0

Mounted at /content/drive


In [ ]:

for i in range(0, len(df_us_bikes)):
  id = df_us_bikes.iloc[i]["networkID"]
  resp = requests.get("http://api.citybik.es/v2/networks/%s?filter=stations" % id)
  data = resp.json()
  for j in data["network"]["stations"]:
    total_bikes = 0
    if j["empty_slots"] != None:
      total_bikes += j["empty_slots"] 
    if j["free_bikes"] != None:
      total_bikes += j["free_bikes"]
    num_bikes += total_bikes
 
print(num_bikes)


131043


There are 741,933 bikes in the bike share program across the US.

# Question 3

You have just finished touring Coit Tower in San Francisco, which is located at latitude 37.802747 and longitude -122.405861. Using your `DataFrame` from Question 2, find the nearest bike station with an available bike, based on taxicab distance (a.k.a. Manhattan distance).

_Hint:_ You can check your answer using Google Maps!

In [ ]:
df_us_bikes["Distance"] = ''

x = 37.802747
y = -122.405861

for i in range(len(df_us_bikes)):
  x1 = df_us_bikes.iloc[i]["latitude"]
  y1 = df_us_bikes.iloc[i]["longitude"]
  
  df_us_bikes.iloc[i, df_us_bikes.columns.get_loc('Distance')] = abs(x-x1) + abs(y - y1)

df_us_bikes['Distance'] = pd.to_numeric(df_us_bikes['Distance'])
min_distance_index = df_us_bikes["Distance"].idxmin()
id = df_us_bikes.loc[min_distance_index]["networkID"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [ ]:

resp = requests.get("http://api.citybik.es/v2/networks/%s?filter=stations" % id)
data = resp.json()

for station in data["network"]["stations"]:
  if (station["free_bikes"] != None and station["free_bikes"] >= 1):
    print(station["name"]) 
    break

Harmon St at Adeline St


Harmon St at Adeline St is the nearest bike station with available bikes from Coit Tower, SF.

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6A - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.